In [ ]:
from collections import namedtuple
import numpy as np
from functools import wraps, partial
from time import time
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 10.0)
from abc import ABC, abstractmethod
from itertools import combinations, product

In [ ]:
import os
import multiprocessing as mp
import concurrent.futures
import fcntl

In [ ]:
Item = namedtuple("Item", ['index', 'value', 'weight'])
LogKey = namedtuple("LogKey", ["method", "estimation_method", "ks_index"])
LogEntry = namedtuple("LogEntry", ["pid", "time", "cache_size", "opt_value", "picked_items"])
HeapNode = namedtuple("HeapNode", ["assignment", "assignment_depth", "value"])

In [ ]:
# log_filepath = "/mnt/c/Users/Shreyas/Desktop/knapsack_opt_logs/session_"+str(int(time()))
log_filepath = "/mnt/c/Users/Shreyas/Desktop/knapsack_opt_logs/session_tests"
def debug_print(*args, debug=False):
    if debug:
        with open(log_filepath, "a+") as log:
            # does python provide a write lock?
            log.write(*args)
            log.write("\n")
            log.flush()
            os.fsync(log.fileno())

In [ ]:
def get_x_axis(k):
    try:
        return float(k[k.index("ks_")+3:].replace("_", "."))
    except ValueError:
        if k == "ks_lecture_dp_1":
            return -1
        else:
            return -2

In [ ]:
class MetricsLogger:
    def __init__(self, metadata=None):
        self.log = {}
        self.metadata = metadata
    def add_log(self, ks_index, t, cache_size, opt_value, picked_items):
        self.log[ks_index] = LogEntry(t, cache_size, opt_value, picked_items)
    def plot_log(self):
        sorted_keys = sorted(self.log.keys(), key=lambda k: get_x_axis(k))
        x_axis = [get_x_axis(k) for k in sorted_keys]
        plt.figure()
        plt.title("Time taken")
        plt.plot(x_axis, [self.log[k].time for k in sorted_keys])
        plt.figure()
        plt.title("Cache size")
        plt.plot(x_axis, [self.log[k].cache_size for k in sorted_keys])
        plt.show()

In [ ]:
def plot_logs(loggers, plot_cache=False, log_scale_x_axis=False):
    if plot_cache:
        plots = ("time_taken", "cache_size", "opt_value")
    else:
        plots = ("time_taken", "opt_value")
    for y in plots:
        plt.figure()
        plt.title(y)
        for name, logger in loggers:
            log = logger.log
            sorted_keys = sorted(log.keys(), key=lambda k: get_x_axis(k))
            x_axis = [get_x_axis(k) for k in sorted_keys]
            if log_scale_x_axis:
                x_axis = np.log(x_axis)
            if y == "time_taken":
                plt.plot(x_axis, [log[k].time for k in sorted_keys], label=name)
            elif y == "cache_size":
                plt.plot(x_axis, [log[k].cache_size for k in sorted_keys], label=name)
            elif y == "opt_value":
                plt.plot(x_axis, [log[k].opt_value for k in sorted_keys], label=name)
        plt.legend()
        plt.show()

In [ ]:
class Logger:
    def __init__(self, metadata=None):
        self.log = {}
        self.metadata = metadata
    def add_log(self, method, estimation_method, ks_index, time, cache_size, opt_value, picked_items):
        self.log[LogKey(method, estimation_method, ks_index)] = (LogEntry(os.getpid(), time, cache_size, opt_value, picked_items))

In [ ]:
def plot_log(log, methods, estimation_methods, log_scale_x_axis=False):
    y_plot = ("time", "cache_size", "opt_value")
    plots = {
        (method, estimation_method): {
            get_x_axis(key.ks_index): log[key] for key in log.keys()
            if method == key.method and estimation_method == key.estimation_method
        }
        for method, estimation_method in product(methods, estimation_methods) 
    }
#     for method, estimation_method in product(methods, estimation_methods):
#         plots[(method, estimation_method)] = {
#             get_x_axis(key.ks_index): log[key] for key in log.keys()
#             if method == key.method and estimation_method == key.estimation_method
#         }
#     for method, estimation_method in product(methods, estimation_methods):
#         for key in log.keys():
#             if method == key.method and estimation_method == key.estimation_method:
#                 plots[(method, estimation_method)].update({get_x_axis(key.ks_index): log[key]})
    for y in y_plot:
        plt.figure()
        plt.title(y)
        for method, estimation_method in plots.keys():
            current_plot = plots[method, estimation_method]
            x_axis = sorted(current_plot.keys())
#             print(list(zip(x_axis, (current_plot[x] for x in x_axis))))
            y_axis = [getattr(current_plot[x], y) for x in x_axis]
            if log_scale_x_axis:
                x_axis = np.log10(x_axis)
            plt.plot(x_axis, y_axis, label=method+"/"+estimation_method)
        plt.legend()
        plt.savefig("/mnt/c/Users/Shreyas/Desktop/knapsack_opt_logs/testing"+y+".png")
        plt.show()

In [ ]:
np.log10([2, 4])

In [ ]:
def timed(f):
    @wraps(f)
    def timed_run(*args, **kwargs):
        start = time()
        y = f(*args, **kwargs)
        end = time()
        delta = end - start
        kwargs["logger"].add_log(kwargs["method"], kwargs["estimation_method"], kwargs["ks_index"], delta, len(y.cache), y.value, y.picked_items)
        return y
    return timed_run

In [ ]:
class Solver(ABC):
    def __init__(self, items, capacity, item_count, ks_index, order, debug):
        if order == "descending_value_density":
            self.items = Solver.value_density_order(items, desc=True)
        elif order == "ascending_value_density":
            self.items = Solver.value_density_order(items)
        elif order == "descending_weight":
            self.items = Solver.weight_order(items, desc=True)
        elif order == "ascending_weight":
            self.items = Solver.weight_order(items)
        else:
            self.items = items
        debug_print("ordered items: {}".format(self.items), debug=debug)
        self.capacity = capacity
        self.item_count = item_count
        self.ks_index = ks_index
        self.value = None
        self.debug = debug
        self.picked_items = []
        self.picked_items_assignment = [0]*self.item_count
        self.cache = {}
    
    @staticmethod
    def value_density_order(items, desc=False):
        return sorted(items, key=lambda item: item.value/item.weight, reverse=desc)
    
    @staticmethod
    def weight_order(items, desc=False):
        return sorted(items, key=lambda item: item.weight, reverse=desc)

    @abstractmethod
    def solve(self, *args, **kwargs):
        pass
    
    def get_picked_items_assignment(self):
        for i in self.picked_items:
            self.picked_items_assignment[i] = 1

In [ ]:
class BoundEstimate:
    def __init__(self):
        pass
    
    @staticmethod
    def is_estimate_within_bounds(estimate, current_max):
        return (current_max - estimate) <= 0.1 * current_max
    
    @staticmethod
    def get_optimistic_assignment(assignment, item_count):
        return assignment + [1 for _ in range(item_count - len(assignment))]
    
    @staticmethod
    def greedy_estimate(items, capacity, item_count, assignment, estimate=None, fractional_items=True):
        assignment = BoundEstimate.get_optimistic_assignment(assignment, item_count)
        debug_print("optimistic assignment is: {}".format(assignment), debug=False)
        if estimate is None:
            estimate = 0
        for i in range(len(items)):
            if capacity <= 0:
                break
            if assignment[i] == 0:
                continue
            if fractional_items:
                capacity_ratio = min(capacity * 1.0/items[i].weight, 1.0)
                estimate += items[i].value * capacity_ratio
                capacity -= items[i].weight * capacity_ratio
            else:
                if items[i].weight > capacity:
                    continue
                else:
                    estimate += items[i].value
                    capacity -= items[i].weight       
        return estimate
    
    @staticmethod
    def unconstrained_estimate(items, capacity, item_count, assignment, estimate=None):
        assignment = BoundEstimate.get_optimistic_assignment(assignment, item_count)
        if estimate is None:
            estimate = 0
        for i in range(len(items)):
            if assignment[i] == 0:
                continue
            else:
                estimate += items[i].value
        return estimate

In [ ]:
class Heap:
    def __init__(self):
        self.heap = []
        self.heap_size = 0
    
    @staticmethod
    def left_child(n):
        return 2*n + 1
    
    @staticmethod
    def right_child(n):
        return 2*n + 2
    
    @staticmethod
    def parent(n):
        return int((n-1)/2)
    
    @staticmethod
    def is_leaf(n, heap_size):
        return Heap.left_child(n) >= heap_size
    
    @staticmethod
    def has_right(n, heap_size):
        return Heap.right_child(n) < heap_size
    
    def is_empty(self):
        return self.heap_size == 0

    def swap(self, x, y):
        temp = self.heap[x]
        self.heap[x] = self.heap[y]
        self.heap[y] = temp

    def add_to_heap(self, node):
        self.heap_size += 1
        self.heap.append(node)
        i = self.heap_size - 1
        # float up
        while i > 0 and self.heap[i].value > self.heap[Heap.parent(i)].value:
            self.swap(i, Heap.parent(i))
            i = Heap.parent(i)
    
    def heapify(self, i):
        if Heap.is_leaf(i, self.heap_size):
            return
        child = None
        left = Heap.left_child(i)
        if not Heap.has_right(i, self.heap_size):
            if self.heap[i].value < self.heap[left].value:
                child = left
        else:
            right = Heap.right_child(i)
            if self.heap[i].value < self.heap[left].value or self.heap[i].value < self.heap[right].value:
                child = left if self.heap[left].value > self.heap[right].value else right
        if child is not None:
            self.swap(i, child)
            self.heapify(child)

    def get_heap_max(self):
        heap_max = self.heap[0]
        self.swap(0, self.heap_size-1)
        self.heap = self.heap[:-1]
        self.heap_size -= 1
        self.heapify(0)
        return heap_max

In [ ]:
class BranchAndBoundSolver(Solver):
    def __init__(
        self, items, capacity, item_count, ks_index,
        search_strategy="depth_first_bb", estimation_method="linear_relaxation", order="descending_value_density",
        debug=False
    ):
        if estimation_method == "linear_relaxation":
            order = "descending_value_density"
            self.get_estimate = BoundEstimate.greedy_estimate
        elif estimation_method == "greedy_estimate":
            self.get_estimate = BoundEstimate.greedy_estimate
        elif estimation_method == "feasible_greedy_estimate":
            self.get_estimate = partial(BoundEstimate.greedy_estimate, fractional_items=False)
        elif estimation_method == "unconstrained_estimate":
            self.get_estimate = BoundEstimate.unconstrained_estimate
        if search_strategy == "depth_first_bb":
            self.search_strategy = self.depth_first_search
        elif search_strategy == "best_first_bb":
            self.search_strategy = self.best_first_search
        elif search_strategy == "lds_bb":
            self.search_strategy = self.least_discrepancy_search
        super().__init__(items, capacity, item_count, ks_index, order, debug)
        self.running_max_estimate = 0
        self.running_max_assignment = None
        self.running_max_assignment_depth = 0
    
    def is_feasible_assignment(self, assignment, assignment_depth):
        weights = [self.items[i].weight for i in range(assignment_depth)]
        assignment_weight = np.inner(weights, assignment)
        return assignment_weight <= self.capacity
        
    def depth_first_search(self):
        stack = []
        stack.append([0])
        stack.append([1])

        while True:
            try:
                assignment = stack.pop()
                assignment_depth = len(assignment)
                debug_print("dfs: current assignment is: {}".format(assignment), debug=self.debug)
                if self.is_feasible_assignment(assignment, assignment_depth):
                    estimate = self.get_estimate(self.items, self.capacity,self.item_count, assignment)
                    debug_print("dfs: optimistic estimate is: {}".format(estimate), debug=self.debug)
                    if assignment_depth <= self.running_max_assignment_depth and estimate <= self.running_max_estimate:
                        debug_print("discarding assignment with estimate {}, while running_max_assignment is {} with estimate {}".format(assignment, estimate, self.running_max_assignment, self.running_max_estimate), debug=self.debug)
                        continue
                    else:
                        if assignment_depth >= self.running_max_assignment_depth:
                            self.running_max_estimate = estimate
                            self.running_max_assignment = assignment
                            self.running_max_assignment_depth = assignment_depth
                        if assignment_depth < self.item_count:
                            stack.append(assignment + [0])
                            stack.append(assignment + [1])
                            debug_print("dfs: after adding nodes, stack is: {}".format(stack), debug=self.debug)
                else:
                    debug_print("assignment is infeasible", debug=self.debug)
                        
            except IndexError:
                break
           
    def best_first_search(self):
        heap = Heap()
        for assignment in [[0], [1]]:
            assignment_depth = len(assignment)
            if self.is_feasible_assignment(assignment, assignment_depth):
                heap.add_to_heap(
                    HeapNode(assignment, assignment_depth, self.get_estimate(self.items, self.capacity, self.item_count, assignment))
                )

        while not heap.is_empty():
            node = heap.get_heap_max()
            assignment = node.assignment
            debug_print("bfs: current assignment is: {}".format(assignment), debug=self.debug)
            assignment_depth = node.assignment_depth
            estimate = node.value
            if assignment_depth <= self.running_max_assignment_depth and estimate <= self.running_max_estimate:
                debug_print("discarding assignment with estimate {}, while running_max_assignment is {} with estimate {}".format(assignment, estimate, self.running_max_assignment, self.running_max_estimate), debug=self.debug)
                continue
            else:
                if assignment_depth >= self.running_max_assignment_depth:
                    self.running_max_estimate = estimate
                    self.running_max_assignment = assignment
                    self.running_max_assignment_depth = assignment_depth
                if assignment_depth < self.item_count:
                    for child in [[0], [1]]:
                        new_assignment = assignment + child
                        new_assignment_depth = assignment_depth + 1
                        debug_print("bfs: new assignment is: {}".format(new_assignment), debug=self.debug)
                        if self.is_feasible_assignment(new_assignment, new_assignment_depth):
                            heap.add_to_heap(
                                HeapNode(new_assignment, new_assignment_depth, self.get_estimate(self.items, self.capacity, self.item_count, new_assignment))
                            )
                        else:
                            debug_print("bfs: new assignment is infeasible", debug=self.debug)
                    debug_print("bfs: after adding nodes, heap is: {}".format(heap.heap), debug=self.debug)

    def _lds_probe(self, assignment, assignment_depth, discrepancies):
        debug_print("lds: assignment is: {}".format(assignment), debug=self.debug)
        if not self.is_feasible_assignment(assignment, assignment_depth):
            debug_print("lds: assignment is infeasible", debug=self.debug)
            self.cache[tuple(assignment)] = False
            return

        try:
            if self.cache[tuple(assignment)] == False:
                debug_print("lds: assignment not explored", debug=self.debug)
                return
        except KeyError:
            self.cache[tuple(assignment)] = self.get_estimate(self.items, self.capacity, self.item_count, assignment)
        finally:
            assignment_estimate = self.cache[tuple(assignment)]
            if assignment_depth < self.running_max_assignment_depth and assignment_estimate < self.running_max_estimate:
                # can't eliminate equal depth nodes because we repeatedly traverse the same nodes
                debug_print(
                    "lds: discarding assignment, assignment_estimate is {}, while running max is {}"
                    .format(assignment_estimate, self.running_max_estimate),
                    debug=self.debug
                )
                self.cache[tuple(assignment)] = False
                return
            elif assignment_depth > self.running_max_assignment_depth or (assignment_depth == self.running_max_assignment_depth and assignment_estimate >= self.running_max_estimate):
                debug_print(
                    "lds: updating max estimates, assignment_estimate is {} at depth {}, while running max is {} at depth {}"
                    .format(assignment_estimate, assignment_depth, self.running_max_estimate, self.running_max_assignment_depth),
                    debug=self.debug
                )
                self.running_max_estimate = assignment_estimate
                self.running_max_assignment = assignment
                self.running_max_assignment_depth = assignment_depth
            if assignment_depth+1 <= self.item_count:
                self._lds_probe(assignment+[1], assignment_depth+1, discrepancies)
                if discrepancies > 0:
                    self._lds_probe(assignment+[0], assignment_depth+1, discrepancies-1)

    def least_discrepancy_search(self):
        for wave_number in range(self.item_count+1):
            debug_print("lds: wave_number is {}".format(wave_number), debug=self.debug)
            self._lds_probe([1], 1, wave_number)
            if wave_number > 0:
                self._lds_probe([0], 1, wave_number-1)
        self.running_max_assignment += [0]*(self.item_count - self.running_max_assignment_depth)

    def solve(self):
        self.search_strategy()
        self.value = self.running_max_estimate
        self.picked_items = [self.items[i].index for i in range(self.item_count) if self.running_max_assignment[i] == 1]
        debug_print("final assignment: {}".format(self.running_max_assignment), debug=self.debug)
        debug_print("picked items: {}".format(self.picked_items), debug=self.debug)
        self.get_picked_items_assignment()


In [ ]:
class NaiveDPSolver(Solver):
    def __init__(self, items, capacity, item_count, ks_index, order=None, debug=False):
        super().__init__(items, capacity, item_count, ks_index, order, debug)
    
    def O(self, k, j):
        item = self.items[j-1]
        debug_print(k, "\t", j)
        debug_print("item weight: {}".format(item.weight), debug=self.debug)
        try:
            return self.cache[(k, j)]
        except KeyError:
            if j == 0:
                self.cache[(k, j)] = 0
            elif item.weight > k:
                self.cache[(k, j)] = self.O(k, j-1)
            else:
                self.cache[(k, j)] = max(
                    self.O(k, j-1),
                    item.value + self.O(k-item.weight, j-1)
                )
        finally:
            return self.cache[(k, j)]
    
    def backtrack(self, k, j):
        if j == 0:
            return
        item = self.items[j-1]
        if self.cache[(k, j)] != self.cache[(k, j-1)]:
            self.picked_items.append(item.index)
            self.backtrack(k-item.weight, j-1)
        else:
            self.backtrack(k, j-1)
    
    def solve(self):
        self.value = self.O(self.capacity, self.item_count)
        self.backtrack(self.capacity, self.item_count)
        self.get_picked_items_assignment()

In [ ]:
naive_dp_log = MetricsLogger(metadata={"solver": "NaiveDP", "order": "none"})
descending_value_density_dp_log = MetricsLogger(metadata={"solver": "NaiveDP", "order": "descending_value_density"})
descending_weight_dp_log = MetricsLogger(metadata={"solver": "NaiveDP", "order": "descending_weight"})
ascending_weight_dp_log = MetricsLogger(metadata={"solver": "NaiveDP", "order": "ascending_weight"})
depth_first_bb_log = MetricsLogger(metadata={"solver": "DepthFirstBB", "order": "descending_value_density"})
best_first_bb_log = MetricsLogger(metadata={"solver": "BestFirstBB", "order": "descending_value_density"})
lds_bb_log = MetricsLogger(metadata={"solver": "LeastDiscrepancyBB", "order": "descending_value_density"})
logger = Logger()

In [ ]:
@timed
def solve(items, capacity, item_count, method, order, ks_index, logger, estimation_method=None, debug=False):
    if "dp" in method:
        solver = NaiveDPSolver(
            items, capacity, item_count, ks_index,
            order=order,
            debug=debug
        )
    elif "bb" in method:
        solver = BranchAndBoundSolver(
            items, capacity, item_count, ks_index,
            search_strategy=method, estimation_method=estimation_method, order=order,
            debug=debug
        )
    solver.solve()
    return solver

In [ ]:
def parse_input_and_solve(file_location, method, estimation_method=None, order=None, debug=False):
    # Modify this code to run your optimization algorithm
    # parse the input
    with open(file_location, 'r') as input_data_file:
        input_data = input_data_file.read()
    ks_index = file_location[file_location.index("ks_"):]
    lines = input_data.split('\n')

    firstLine = lines[0].split()
    item_count = int(firstLine[0])
    capacity = int(firstLine[1])

    items = []

    for i in range(1, item_count+1):
        line = lines[i]
        parts = line.split()
        items.append(Item(i-1, int(parts[0]), int(parts[1])))
    
#     logger = Logger()

    return solve(
        items, capacity, item_count,
        method=method, estimation_method=estimation_method,
        order=order, ks_index=ks_index, logger=logger,
        debug=debug
    ), logger

def solve_it(file_location, method, estimation_method=None, order=None, debug=False):
    solved, logger = parse_input_and_solve(file_location, method, estimation_method, order, debug)
    taken = solved.picked_items_assignment
    debug_print("picked items: {}".format(solved.picked_items), debug=debug)
    debug_print("picked items assignment: {}".format(solved.picked_items_assignment), debug=debug)
    # prepare the solution in the specified output format
    output_data = str(int(solved.value)) + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, taken))

    return output_data, logger

In [ ]:
for test in ["ks_19_0"]:
    file_location = "/mnt/c/Users/Shreyas/Desktop/discrete_optimization/knapsack/data/" + test
    print("solving for test case: ", test)
    print(solve_it(file_location, method="lds_bb", estimation_method="feasible_greedy_estimate", order="descending_value_density", debug=False))
#     print("solving depth_first")
#     print(solve_it(file_location, method="depth_first_bb", estimation_method="linear_relaxation", order="descending_value_density", debug=True))

In [ ]:
logger.log

In [ ]:
plot_log(logger.log, methods=["lds_bb"], estimation_methods=["feasible_greedy_estimate"], log_scale_x_axis=True)

In [ ]:
def run_tests():
    test_cases = [
        'ks_19_0',
        'ks_30_0',
        'ks_40_0',
        'ks_45_0',
        'ks_4_0',
        'ks_50_0',
        'ks_50_1',
        'ks_60_0',
        'ks_82_0',
        'ks_100_0',
        'ks_100_1',
        'ks_100_2',
        'ks_106_0',
        'ks_200_0',
        'ks_200_1',
        'ks_300_0',
        'ks_400_0',
        'ks_500_0',
        'ks_1000_0',
        'ks_10000_0'
    ]
    methods = ["best_first_bb", "lds_bb"]
    estimation_methods = ["feasible_greedy_estimate"]
    orders = ["descending_value_order"]
    main_logger = {}
    with concurrent.futures.ProcessPoolExecutor(mp.cpu_count()*2) as executor:
        futures = {
            executor.submit(
                solve_it,
                file_location="/mnt/c/Users/Shreyas/Desktop/discrete_optimization/knapsack/data/" + test,
                method=method,
                estimation_method=estimation_method,
                order=order,
                debug=False
            ): (test, method, estimation_method, order)
            for test, method, estimation_method, order in product(
                test_cases, methods, estimation_methods, orders
            )
        }
        for future in concurrent.futures.as_completed(futures):
            output, logger = future.result()
            print("Completed ", futures[future])
            main_logger.update(logger.log)
    plot_log(main_logger, methods, estimation_methods, log_scale_x_axis=True)
    return main_logger
main_logger = run_tests()

In [ ]:
plomain_logger

In [ ]:
best_first_bb_log.log

In [ ]:
for test in test_cases:
    file_location = "/mnt/c/Users/Shreyas/Desktop/discrete_optimization/knapsack/data/" + test
    print("solving for test case: ", test)
    print("solving best_first")
    print(solve_it(file_location, method="best_first_bb", estimation_method="linear_relaxation", order="descending_value_density", debug=True))

In [ ]:
file_location = "/mnt/c/Users/Shreyas/Desktop/discrete_optimization/knapsack/data/" + "ks_4_0"
print(solve_it(file_location, method="lds_bb", estimation_method="linear_relaxation", order="descending_value_density"))
print(solve_it(file_location, method="depth_first_bb", estimation_method="linear_relaxation", order="descending_value_density"))

In [ ]:
for test in test_cases:
    file_location = "/mnt/c/Users/Shreyas/Desktop/discrete_optimization/knapsack/data/" + test
    solve_it(file_location, method="dp", order="ascending_weight")
    solve_it(file_location, method="dp", order="descending_weight")
    solve_it(file_location, method="dp", order="descending_value_density")
    solve_it(file_location, method="dp", order=None)

In [ ]:
plot_logs([
    ("depth_first", depth_first_bb_log),
    ("best_first", best_first_bb_log),
    ("lds", lds_bb_log),
], log_scale_x_axis=True)

In [ ]:
plot_logs([
    ("descending_weight", descending_weight_dp_log),
    ("descending_value_density", descending_value_density_dp_log),
    ("ascending_weight", ascending_weight_dp_log),
    ("naive", naive_dp_log)
])